In [2]:
import pandas as pd
import numpy as np
import os
import acquire_nick as get
import prepare_nick as prep
import re

In [9]:
df = get.acquire_data()

Using cached CSV


In [10]:
df = prep.prep_data(df)

clean.csv not detected.
processing capstone.csv
clean.csv ready for future use


In [70]:
# Deleted columns Unnamed: 0
df = df.drop(['Unnamed: 0'], axis=1)


In [14]:
df.head(1)

,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,TomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette,JohnLasseter JossWhedon AndrewStanton JoelCohen AlecSokolow BonnieArnold EdCatmull RalphGuggenheim SteveJobs LeeUnkrich RalphEggleston RobertGordon MaryHelenLeasman KimBlanchette MarilynMcCoppen RandyNewman DaleE.Grahn RobinCooper JohnLasseter PeteDocter JoeRanft PatsyBouge NormDeCarlo AshBrannon RandyNewman RomanFigun DonDavis JamesFlamberg MaryBethSmith RickMackay SusanBradley WilliamReeves RandyNewman AndrewStanton PeteDocter GaryRydstrom KarenRobertJackson ChrisMontan RichQuade MichaelBerenstein ColinBrady DaveyCrockettFeiten AngieGlocka RexGrignon TomK.Gurney JimmyHayward HalT.Hickel KarenKiser AnthonyB.LaMolinara GuionneLeroy BudLuckey LesMajor GlennMcQueen MarkOftedal JeffPidgeon JeffPratt SteveRabatich RogerRose SteveSegal DougSheppeck AlanSperling DougSweetland DavidTart KenWillard ThomasPorter MarkThomasHenne OrenJacob DarwynPeachey MitchPrater BrianM.Rosen SharonCalahan GalynSusman WilliamCone ShelleyDanielsLekven BobPauley BudLuckey AndrewStanton WilliamCone SteveJohnson DanHaskett TomHolloway JeanGillmore DesiréeMourad KellyO'Connell SonokoKonishi AnnM.Rockwell JulieM.McDonald RobinLee TomFreeman AdaCochavi DanaMulligan DeirdreMorrison LoriLombardo EllenDevine LaurenBethStrogoff GaryRydstrom GarySummers TimHolland PatJackson TomMyers J.R.Grubbs SusanSanford SusanPopovic DanEngstrom RuthLambert MickieMcGowan,862,Toy Story,Animation Comedy Family,30000000,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,PixarAnimationStudios,UnitedStatesofAmerica,373554033.0,81.0,7.7,5415.0,jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life,1995-10-30


Plans:
Extract the number of names inside of crew and create a column that contains a count of names that appear. Consider the same for cast.     

Scaling will be needed for the differences in scope among votes, popularity, budget, revenue, et cetera.

I'm curious as to whether or not number of keywords, or the types of keywords themselves will have a signifcant influence on the predictability of scores or budget. Presumably the film studio will have that effect, and the production company may also. 

ID should probably be cast as a category, or at the very least a generic string, since there is no rank or meaning of the order per id. 

Release date could be converted to a pandas date-time format for the sake of time-series analysis. Otherwise, it seems much of what we will be doing going forward can be framed under many different methodologies. 

In [17]:
prep.describe_data(df)

This dataframe has 7457 rows and 16 columns.

--------------------------------------
--------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7457 entries, 0 to 7456
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cast                  7423 non-null   object 
 1   crew                  7444 non-null   object 
 2   id                    7457 non-null   int64  
 3   title                 7457 non-null   object 
 4   genres                7434 non-null   object 
 5   budget                7457 non-null   int64  
 6   overview              7435 non-null   object 
 7   popularity            7457 non-null   float64
 8   production_companies  7039 non-null   object 
 9   production_countries  7301 non-null   object 
 10  revenue               7457 non-null   float64
 11  runtime               7451 non-null   float64
 12  vote_average          7457 non-null   float64
 13 

TypeError: '<=' not supported between instances of 'str' and 'float'

Because the total proportion of the data frame affected by nulls is barely one percent, and since the columns containing these values can not be inferred due to their qualitative nature that varies from film to film, dropping them from the data frame altogether is perfectly acceptable. 

In [71]:
df.shape
# first lets see what happens when I remove na's from just the keywords column; ideally, most of the 
df2 = df.copy()
# I will use this at the end for comparison's sake. I will remove all the nas from this data frame at once

In [33]:
df = df.dropna(subset=['keywords'])

In [34]:
prep.nulls(df)

Null Values by Column: 
                      count  percentage
production_companies    236        3.48
production_countries     68        1.00
cast                     17        0.25
overview                 12        0.18
runtime                   4        0.06
crew                      3        0.04
genres                    3        0.04
id                        0        0.00
title                     0        0.00
budget                    0        0.00
popularity                0        0.00
revenue                   0        0.00
vote_average              0        0.00
vote_count                0        0.00
keywords                  0        0.00
release_date              0        0.00
 
 Total Number of Missing Values: 343 

 Proportion of Nulls in Dataframe: 0.0032

--------------------------------------
--------------------------------------
Row-by-Row Nulls
                 percent_cols_missing  num_rows
num_cols_missing                               
1                    

In [35]:
# now with production companies
df = df.dropna(subset=['production_companies'])

In [36]:
prep.nulls(df)

Null Values by Column: 
                      count  percentage
production_countries     19        0.29
overview                 11        0.17
cast                      5        0.08
runtime                   4        0.06
crew                      2        0.03
genres                    1        0.02
id                        0        0.00
title                     0        0.00
budget                    0        0.00
popularity                0        0.00
production_companies      0        0.00
revenue                   0        0.00
vote_average              0        0.00
vote_count                0        0.00
keywords                  0        0.00
release_date              0        0.00
 
 Total Number of Missing Values: 42 

 Proportion of Nulls in Dataframe: 0.0004

--------------------------------------
--------------------------------------
Row-by-Row Nulls
                 percent_cols_missing  num_rows
num_cols_missing                               
1                     

In [37]:
df.shape

(6538, 16)

In [72]:
# now I will drop the rest. 
df = df.dropna()

In [39]:
df.shape

(6502, 16)

In [40]:
df2 = df2.dropna()

In [41]:
df2.shape

(6502, 16)

In [ ]:
# the same result. 

In [47]:
# check for duplicates:
df['duplicate'] = df.duplicated()

In [52]:
dups = df[df['duplicate'] == True]
# at this point I am reminded of the films where budgets are zero. 
# these indicate reissues or re-releases, and in the event we label a film as being successful
# on the basis of revenue == 2x budget, for example, these have an unfair, inaccurate advantage.
# I am removing all films where the budget is equal to zero. Then I will check duplicates again. 
dups.count()
# something is definitely weird about this. 

cast                    53
crew                    53
id                      53
title                   53
genres                  53
budget                  53
overview                53
popularity              53
production_companies    53
production_countries    53
revenue                 53
runtime                 53
vote_average            53
vote_count              53
keywords                53
release_date            53
duplicate               53
dtype: int64

In [74]:
# regardless, I will keep first, where the budget is not zero, and drop the rest. one way to do that 
# is to first resort the dataframe. 
df = df.sort_values(by='budget', ascending=False, na_position='last')
# keep first instance of the duplicate
df.duplicated(keep='first')

5013    False
3972    False
6232    False
3745    False
5270    False
        ...  
953     False
4451    False
4450    False
4449    False
7451    False
Length: 6502, dtype: bool

In [91]:
# in case something goes wrong 
df_safe = df.copy()
# change inplace to True if the results are good, for the prepare function
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)

In [76]:
df.shape
# duplicates are gone. 

(6449, 16)

In [78]:
df[df['budget'] == 0].count()

cast                    1539
crew                    1539
id                      1539
title                   1539
genres                  1539
budget                  1539
overview                1539
popularity              1539
production_companies    1539
production_countries    1539
revenue                 1539
runtime                 1539
vote_average            1539
vote_count              1539
keywords                1539
release_date            1539
dtype: int64

Hold the phone. That's way more than I thought. We may need to actually handle this based on imputation. 

In [77]:
df.shape

(6449, 16)

***
One thing I just realized is that the actors (cast) aren't listed in alphabetical order, meaning that they are cast (most likely) first in terms of prominence, and then as a matter of appearance. There is a way to extract the top 3 actors. But with the way the DF currently is, i will need to collaborate with my teams on that. Worth consideration. 
***

In [81]:
## create year column
## convert release_date to have same format
#for index, row in df.iterrows():
#    try:
#        date = parser.parse(row['release_date'])
#        year = date.year
#        newDate = f'{date.year}-{date.month}-{date.day}'
#    except:     # if release_date is empty
#        year = np.nan
#        newDate = np.nan
#
#    df.at[index, 'year'] = year
#    df.at[index, 'release_date'] = newDate

In [93]:
df = df_safe.copy()

In [95]:
df['release_date'] = df['release_date'].astype('datetime64[ns]')

In [96]:
df.dtypes

cast                            object
crew                            object
id                               int64
title                           object
genres                          object
budget                           int64
overview                        object
popularity                     float64
production_companies            object
production_countries            object
revenue                        float64
runtime                        float64
vote_average                   float64
vote_count                     float64
keywords                        object
release_date            datetime64[ns]
dtype: object

In [97]:
df.release_date

0      2011-05-14
1      2007-05-19
2      2015-04-22
3      2006-06-28
4      2012-03-07
          ...    
6444   1982-05-21
6445   2009-02-08
6446   1992-05-01
6447   2009-02-06
6448   2014-03-13
Name: release_date, Length: 6449, dtype: datetime64[ns]

In [32]:
# testing prepare function
df = prep.prep_data(df)

In [5]:
df.shape

(6449, 16)

In [6]:
test = df[df.release_date >= '2000-01-01']

In [7]:
test.shape

(3851, 16)

In [ ]:
# removing dates based on arbitrarily capturing 'modern cinema' would be too much
# of a loss at this juncture. 

In [15]:
df['budget'].median()

10000000.0

In [16]:
df['budget'].mean()

25213064.972088695

In [17]:
df.budget.sort_values()

6448            0
5415            0
5416            0
5417            0
5418            0
          ...    
4       260000000
3       270000000
2       280000000
1       300000000
0       380000000
Name: budget, Length: 6449, dtype: int64

In [18]:
# df['budget'] = df['budget'].fillna(df['budget'].median())
# zero is not NaN. Silly me. 

In [21]:
df['budget'] = df['budget'].replace(0, df['budget'].median())

In [22]:
df.budget.sort_values()

4909            1
4908            1
4907            1
4906            1
4905            1
          ...    
4       260000000
3       270000000
2       280000000
1       300000000
0       380000000
Name: budget, Length: 6449, dtype: int64

In [25]:
df['budget'] = df['budget'].replace(<= 100000, df['budget'].median())

SyntaxError: invalid syntax (3752837760.py, line 1)

In [28]:
df2 = df.copy()

In [64]:
df2['budget'] = np.where(df['budget'].between(0,1000000), df['budget'].median(), df['budget'])

In [ ]:
# range1 = np.where(df['budget'].between(0,100), df['budget'].median(), df['budget'])
# range2 = np.where(df['budget'].between(101,1000), df['budget'].median(), df['budget'])
# range3 = np.where(df['budget'].between(1001,10000), df['budget'].median(), df['budget'])
# range4 = np.where(df['budget'].between(10001,100000), df['budget'].median(), df['budget'])

In [70]:
#df2['Decile_rank'] = pd.qcut(df2['budget'], 10, labels = False)

In [66]:
df2.budget.median() 

10000000.0

In [69]:
df2.head()

,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,johnnydepp penélopecruz ianmcshane kevinmcnally geoffreyrush samclaflin astridbergès-frisbey stephengraham gregellis damiano'hare óscarjaenada richardgriffiths keithrichards gemmaward judidench ianmercer robbiekay christopherfairbank yukimatsuzaki steveevets bronsonwebb derekmears deobiaoparei sebastianarmesto antonlesser rogerallam paulbazely richardthomson dannyleboyer alanj.utley-moore lukeroberts emiliajones sanyahughes danielings lejon siegfriedpeters,dariuszwolski jerrybruckheimer tedelliott terryrossio terryrossio hanszimmer mikestenson chadoman christopherboyes pennyrose bobbadami davidbrenner shannonmills deeselby barryh.waldman gordonsim johnmyhre robmarshall paulmassey kevinkaska veneill johndeluca missyparker michaelhertlein margitpfeiffer kerrylynmckissick wyattsmith melissamuik melissamuik georgewattersii petermountain davidc.hughes barbaramcdermott kenfischer adamkopald aaronwilliams elizabethmcclurg katialewinjablonsky leeorloff,1865,pirates of the caribbean: on stranger tides,adventure action fantasy,380000000.0,"captain jack sparrow crosses paths with a woman from his past, and he's not sure if it's love -- or if she's a ruthless con artist who's using him to find the fabled fountain of youth. when she forces him aboard the queen anne's revenge, the ship of the formidable pirate blackbeard, jack finds himself on an unexpected adventure in which he doesn't know who to fear more: blackbeard or the woman from his past.",27.887720,waltdisneypictures jerrybruckheimerfilms movingpicturecompany(mpc),unitedstatesofamerica,1.045714e+09,136.0,6.4,5068.0,sea captain mutiny sword prime minister sailing silver ship duke mermaid pirate soldier battle swashbuckler aftercreditsstinger 3d,2011-05-14
1,johnnydepp orlandobloom keiraknightley stellanskarsgård chowyun-fat billnighy geoffreyrush jackdavenport kevinmcnally tomhollander naomieharris jonathanpryce keithrichards leearenberg mackenziecrook gregellis davidbailie martinklebba davidschofield laurenmaher vanessabranch angusbarnett gilesnew reggielee dominicscottkay takayofischer davidmeunier ho-kwantse andybeckwith peterdonaldbadalamentiii christophers.capp keithrichards hakeemkae-kazim ghassanmassoud,dariuszwolski goreverbinski jerrybruckheimer tedelliott terryrossio stephene.rivkin craigwood hanszimmer mikestenson ericmcleod chadoman peterkohn patsandston denisechamian rickheinrichs johndexter priscillajohn cherylcarasik lizdann pennyrose bobbadami jameswardbyrkit veneill johndixon dottiestarling pamelaalch allenhall melissamuik sharronreynolds barbaramcdermott karengolden katiegreathouse,285,pirates of the caribbean: at world's end,adventure fantasy action,300000000.0,"captain barbossa, long believed to be dead, has come back to life and is headed to the edge of the earth with will turner and elizabeth swann. but nothing is quite as it seems.",31.363664,waltdisneypictures jerrybruckheimerfilms secondmateproductions,unitedstatesofamerica,9.610000e+08,169.0,6.9,4627.0,ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger,2007-05-19
2,robertdowneyjr. chrishemsworth markruffalo chrisevans scarlettjohansson jeremyrenner jamesspader samuell.jackson doncheadle aarontaylor-johnson elizabetholsen paulbettany cobiesmulders anthonymackie hayleyatwell idriselba lindacardellini stellanskarsgård claudiakim thomaskretschmann andyserkis juliedelpy stanlee henrygoodman chrisluca brianschaeffer dominiqueprovost-chalkley isaacandrews garethkieranjones chanwoolim minheeyeo bentleykalu julianbleach christopherbeasley vuyodabula nondumisotembe kabelothai leleleowaba mandlagaduka harrietmanamela beulahhashe muscakumalo mathapeloseptember antonyacheampong chiomaanyanwu bensakamoto imogenpoynton isabellapo

In [22]:
df = prep.prep_data(df)

In [4]:
df

,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,JohnnyDepp PenélopeCruz IanMcShane KevinMcNally GeoffreyRush SamClaflin AstridBergès-Frisbey StephenGraham GregEllis DamianO'Hare ÓscarJaenada RichardGriffiths KeithRichards GemmaWard JudiDench IanMercer RobbieKay ChristopherFairbank YukiMatsuzaki SteveEvets BronsonWebb DerekMears DeobiaOparei SebastianArmesto AntonLesser RogerAllam PaulBazely RichardThomson DannyLeBoyer AlanJ.Utley-Moore LukeRoberts EmiliaJones SanyaHughes DanielIngs LeJon SiegfriedPeters,DariuszWolski JerryBruckheimer TedElliott TerryRossio TerryRossio HansZimmer MikeStenson ChadOman ChristopherBoyes PennyRose BobBadami DavidBrenner ShannonMills DeeSelby BarryH.Waldman GordonSim JohnMyhre RobMarshall PaulMassey KevinKaska VeNeill JohnDeLuca MissyParker MichaelHertlein MargitPfeiffer KerryLynMcKissick WyattSmith MelissaMuik MelissaMuik GeorgeWattersII PeterMountain DavidC.Hughes BarbaraMcDermott KenFischer AdamKopald AaronWilliams ElizabethMcClurg KatiaLewinJablonsky LeeOrloff,1865,Pirates of the Caribbean: On Stranger Tides,Adventure Action Fantasy,380000000.0,"Captain Jack Sparrow crosses paths with a woman from his past, and he's not sure if it's love -- or if she's a ruthless con artist who's using him to find the fabled Fountain of Youth. When she forces him aboard the Queen Anne's Revenge, the ship of the formidable pirate Blackbeard, Jack finds himself on an unexpected adventure in which he doesn't know who to fear more: Blackbeard or the woman from his past.",27.887720,WaltDisneyPictures JerryBruckheimerFilms MovingPictureCompany(MPC),UnitedStatesofAmerica,1.045714e+09,136.0,6.4,5068.0,sea captain mutiny sword prime minister sailing silver ship duke mermaid pirate soldier battle swashbuckler aftercreditsstinger 3d,2011-05-14
1,JohnnyDepp OrlandoBloom KeiraKnightley StellanSkarsgård ChowYun-fat BillNighy GeoffreyRush JackDavenport KevinMcNally TomHollander NaomieHarris JonathanPryce KeithRichards LeeArenberg MackenzieCrook GregEllis DavidBailie MartinKlebba DavidSchofield LaurenMaher VanessaBranch AngusBarnett GilesNew ReggieLee DominicScottKay TakayoFischer DavidMeunier Ho-KwanTse AndyBeckwith PeterDonaldBadalamentiII ChristopherS.Capp KeithRichards HakeemKae-Kazim GhassanMassoud,DariuszWolski GoreVerbinski JerryBruckheimer TedElliott TerryRossio StephenE.Rivkin CraigWood HansZimmer MikeStenson EricMcLeod ChadOman PeterKohn PatSandston DeniseChamian RickHeinrichs JohnDexter PriscillaJohn CherylCarasik LizDann PennyRose BobBadami JamesWardByrkit VeNeill JohnDixon DottieStarling PamelaAlch AllenHall MelissaMuik SharronReynolds BarbaraMcDermott KarenGolden KatieGreathouse,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,300000000.0,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",31.363664,WaltDisneyPictures JerryBruckheimerFilms SecondMateProductions,UnitedStatesofAmerica,9.610000e+08,169.0,6.9,4627.0,ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger,2007-05-19
2,RobertDowneyJr. ChrisHemsworth MarkRuffalo ChrisEvans ScarlettJohansson JeremyRenner JamesSpader SamuelL.Jackson DonCheadle AaronTaylor-Johnson ElizabethOlsen PaulBettany CobieSmulders AnthonyMackie HayleyAtwell IdrisElba LindaCardellini StellanSkarsgård ClaudiaKim ThomasKretschmann AndySerkis JulieDelpy StanLee HenryGoodman ChrisLuca BrianSchaeffer DominiqueProvost-Chalkley IsaacAndrews GarethKieranJones ChanWooLim MinheeYeo BentleyKalu JulianBleach ChristopherBeasley VuyoDabula NondumisoTembe KabeloThai LeleLeowaba MandlaGaduka HarrietManamela BeulahHashe MuscaKumalo MathapeloSeptember AntonyAcheampong ChiomaAnyanwu BenSakamoto ImogenPoynton IsabellaPo

In [5]:
df.budget.sort_values()

4561      1020000.0
4560      1050255.0
4559      1052753.0
4558      1089360.0
4557      1100000.0
           ...     
4       260000000.0
3       270000000.0
2       280000000.0
1       300000000.0
0       380000000.0
Name: budget, Length: 6449, dtype: float64

In [23]:
df2 = df.copy()

In [30]:
# The values were stored in the new column
# df2['profitable'] = np.where((df2['revenue']) >= (df2['budget'] * 2))

In [31]:
# create success column
# movie is successful if (revenue >= budget * 2)
for index, row in df.iterrows():
    try:
        budget = df.at[index, 'budget']
        revenue = df.at[index, 'revenue']
        if (revenue >= budget * 2):
            profitable = True
        else:
            profitable = False
    except:     # if budget or revenue is empty
        profitable = np.nan

    df.at[index, 'profitable'] = profitable

In [36]:
df['profitable'] = df['profitable'].astype(bool)

In [37]:
df.dtypes

cast                            object
crew                            object
id                               int64
title                           object
genres                          object
budget                         float64
overview                        object
popularity                     float64
production_companies            object
production_countries            object
revenue                        float64
runtime                        float64
vote_average                   float64
vote_count                     float64
keywords                        object
release_date            datetime64[ns]
profitable                        bool
dtype: object

In [4]:
df = get.acquire_data()
df = prep.prep_data(df)

Using cached CSV


In [5]:
df.shape

(6503, 19)

In [47]:
re.search(r'[A-Z][a-z]+[A-Z][a-z]+', str(df.cast))

<re.Match object; span=(8, 18), match='JohnnyDepp'>

In [3]:
#re.search(r'[A-Z][a-z]+[A-Z][a-z]+{2}', str(df.cast))

In [ ]:
re.search()

In [ ]:
df

In [ ]:
df4[df4['id'] != '1997-08-20']